# Thermoacoustic stability prediction using multilabel classification algorithms
### Part 1: Models

The objective of this project is to predict the thermoacoustic stability of a combustor in various frequency intervals using multilabel classification algorithms.

---

Data source: Artificial data generated with a modified version of OSCILOS_lite (freely available at https://github.com/MorgansLab/OSCILOS_lite)

---

Author: Renaud Gaudron\
Version: 6.0\
Latest update: 18/08/21

## Step 1: Preprocessing of the artificial data

In [1]:
# Importing the relevant libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from skmultilearn.problem_transform import ClassifierChain
import time

In [2]:
# Opening and printing to log file

sourceFile = open('Multilabel_Classification_log.txt', 'w')

print("----------------------------------------------------------------------------------------------------------------------------\n",file=sourceFile)
print("Thermoacoustic stability prediction using multilabel classification algorithms\n".upper(),file=sourceFile)
print("----------------------------------------------------------------------------------------------------------------------------\n",file=sourceFile)
print("Data source: Artificial data generated with a modified version of OSCILOS_lite (freely available at https://github.com/MorgansLab/OSCILOS_lite)\n",file=sourceFile)
print("Author: Renaud Gaudron\n",file=sourceFile)
print("Version: 6.0\n",file=sourceFile)
print("Latest update: 18/08/21\n",file=sourceFile)
print("----------------------------------------------------------------------------------------------------------------------------\n",file=sourceFile)
print("Step 1 : Preprocessing of the artificial data".upper(),file=sourceFile)

In [3]:
# Loading the data

data = pd.read_csv("Cases_large.txt", sep="\t",usecols=list(range(20))) # Storing data as a Pandas framework

headers=["X0","X1","X2","X3","R0","R1","R2","R3","Gain","Phase","0_50","50_100","100_150","150_200",
         "200_250","250_300","300_350","350_400","400_450","450_500"] 
data.columns = headers # Adding headers to the data

data["0_50"] = data["0_50"].astype('int') # Change type for the column "Unstable" 
data["50_100"] = data["50_100"].astype('int') # Change type for the column "Unstable" 
data["100_150"] = data["100_150"].astype('int') # Change type for the column "Unstable" 
data["150_200"] = data["150_200"].astype('int') # Change type for the column "Unstable" 
data["200_250"] = data["200_250"].astype('int') # Change type for the column "Unstable" 
data["250_300"] = data["250_300"].astype('int') # Change type for the column "Unstable" 
data["300_350"] = data["300_350"].astype('int') # Change type for the column "Unstable" 
data["350_400"] = data["350_400"].astype('int') # Change type for the column "Unstable" 
data["400_450"] = data["400_450"].astype('int') # Change type for the column "Unstable" 
data["450_500"] = data["450_500"].astype('int') # Change type for the column "Unstable" 

# The stability column indicates whether a combustor is practically stable (0) or a practically unstable (1) for each frequency range

# Let's drop columns X0 and R3 as they are redundant
data=data.drop(columns=['X0','R3'])

# X represents the absolute axial position of the element. Let's compute Li=Xi-Xi-1 corresponding to the length of each element.

data['X3']=data['X3']-data['X2']
data['X2']=data['X2']-data['X1']

data.rename(columns={"X1":"L1"}, inplace=True) # We just need to change the heading for this one
data.rename(columns={"X2":"L2"}, inplace=True) # We just need to change the heading for this one
data.rename(columns={"X3":"L3"}, inplace=True) # We just need to change the heading for this one

#data=data[:100] # FOR VALIDATION ONLY - Select the first 100 examples

print("\nTop 10 lines of the pre-processed data:\n",file=sourceFile)
print(round(data.head(10),4).to_markdown(index = False, tablefmt="fancy_grid"),file=sourceFile)

data.head(10) # Display the top 10 lines

,L1,L2,L3,R0,R1,R2,Gain,Phase,0_50,50_100,100_150,150_200,200_250,250_300,300_350,350_400,400_450,450_500
0,0.215282,0.323424,0.170701,0.082752,0.053804,0.010102,0.885752,2.551514,0,0,0,0,0,1,0,0,0,0
1,0.140167,0.188399,0.423352,0.089381,0.089260,0.078452,0.518373,3.134972,0,0,0,0,1,0,0,0,0,1
2,0.166067,0.111135,0.155982,0.023319,0.084748,0.055220,0.615510,2.905676,0,0,0,0,0,0,1,0,0,0
3,0.159184,0.338507,0.276718,0.020077,0.042646,0.056869,0.875384,4.900099,0,0,0,1,0,0,0,0,0,0
4,0.488712,0.281809,0.156784,0.081084,0.035787,0.058883,0.765332,1.543422,0,0,0,0,0,0,1,0,0,0
5,0.229345,0.362074,0.343772,0.060946,0.033539,0.052616,0.829335,3.811601,0,0,0,0,0,0,0,0,1,0
6,0.172424,0.148255,0.525720,0.045043,0.078339,0.020235,0.982497,4.370344,0,0,1,0,0,0,1,0,0,0
7,0.631154,0.050756,0.200507,0.069525,0.018376,0.017569,0.756406,2.116734,0,0,0,0,0,1,0,0,0,0
8,0.511980,0.048393,0.319640,0.035053,0.022520,0.098560,0.733090,4.961723,0,0,1,0,0,0,0,0,1,0
9,0.502112,0.200094,0.108543,0.033783,0.069184,0.019358,0.757796,3.491583,0,0,0,1,1,0,0,0,1,0


In [4]:
data.dtypes # Checking the data types for each column

L1         float64
L2         float64
L3         float64
R0         float64
R1         float64
R2         float64
Gain       float64
Phase      float64
0_50         int64
50_100       int64
100_150      int64
150_200      int64
200_250      int64
250_300      int64
300_350      int64
350_400      int64
400_450      int64
450_500      int64
dtype: object

In [5]:
print("\n{} stable configurations - {} unstable configurations between 0-50 Hz.".format(data["0_50"].value_counts()[0],data["0_50"].value_counts()[1]),file=sourceFile)
print("{} stable configurations - {} unstable configurations between 0-50 Hz.".format(data["0_50"].value_counts()[0],data["0_50"].value_counts()[1]))

print("{} stable configurations - {} unstable configurations between 50-100 Hz.".format(data["50_100"].value_counts()[0],data["50_100"].value_counts()[1]),file=sourceFile)
print("{} stable configurations - {} unstable configurations between 50-100 Hz.".format(data["50_100"].value_counts()[0],data["50_100"].value_counts()[1]))

print("{} stable configurations - {} unstable configurations between 100-150 Hz.".format(data["100_150"].value_counts()[0],data["100_150"].value_counts()[1]),file=sourceFile)
print("{} stable configurations - {} unstable configurations between 100-150 Hz.".format(data["100_150"].value_counts()[0],data["100_150"].value_counts()[1]))

print("{} stable configurations - {} unstable configurations between 150-200 Hz.".format(data["150_200"].value_counts()[0],data["150_200"].value_counts()[1]),file=sourceFile)
print("{} stable configurations - {} unstable configurations between 150-200 Hz.".format(data["150_200"].value_counts()[0],data["150_200"].value_counts()[1]))

print("{} stable configurations - {} unstable configurations between 200-250 Hz.".format(data["200_250"].value_counts()[0],data["200_250"].value_counts()[1]),file=sourceFile)
print("{} stable configurations - {} unstable configurations between 200-250 Hz.".format(data["200_250"].value_counts()[0],data["200_250"].value_counts()[1]))

print("{} stable configurations - {} unstable configurations between 250-300 Hz.".format(data["250_300"].value_counts()[0],data["250_300"].value_counts()[1]),file=sourceFile)
print("{} stable configurations - {} unstable configurations between 250-300 Hz.".format(data["250_300"].value_counts()[0],data["250_300"].value_counts()[1]))

print("{} stable configurations - {} unstable configurations between 300-350 Hz.".format(data["300_350"].value_counts()[0],data["300_350"].value_counts()[1]),file=sourceFile)
print("{} stable configurations - {} unstable configurations between 300-350 Hz.".format(data["300_350"].value_counts()[0],data["300_350"].value_counts()[1]))

print("{} stable configurations - {} unstable configurations between 350-400 Hz.".format(data["350_400"].value_counts()[0],data["350_400"].value_counts()[1]),file=sourceFile)
print("{} stable configurations - {} unstable configurations between 350-400 Hz.".format(data["350_400"].value_counts()[0],data["350_400"].value_counts()[1]))

print("{} stable configurations - {} unstable configurations between 400-450 Hz.".format(data["400_450"].value_counts()[0],data["400_450"].value_counts()[1]),file=sourceFile)
print("{} stable configurations - {} unstable configurations between 400-450 Hz.".format(data["400_450"].value_counts()[0],data["400_450"].value_counts()[1]))

print("{} stable configurations - {} unstable configurations between 450-500 Hz.\n".format(data["450_500"].value_counts()[0],data["450_500"].value_counts()[1]),file=sourceFile)
print("{} stable configurations - {} unstable configurations between 450-500 Hz.".format(data["450_500"].value_counts()[0],data["450_500"].value_counts()[1]))

548203 stable configurations - 3154 unstable configurations between 0-50 Hz.
522296 stable configurations - 29061 unstable configurations between 50-100 Hz.
476755 stable configurations - 74602 unstable configurations between 100-150 Hz.
470435 stable configurations - 80922 unstable configurations between 150-200 Hz.
497355 stable configurations - 54002 unstable configurations between 200-250 Hz.
494253 stable configurations - 57104 unstable configurations between 250-300 Hz.
467536 stable configurations - 83821 unstable configurations between 300-350 Hz.
461529 stable configurations - 89828 unstable configurations between 350-400 Hz.
470144 stable configurations - 81213 unstable configurations between 400-450 Hz.
472000 stable configurations - 79357 unstable configurations between 450-500 Hz.


In [6]:
# Computing the Kendall correlation coefficients

Kendall_corr=data.corr(method='kendall') # Compute the correlation between the different variables
Kendall_corr.to_csv(r'Kendall_correlations_Multilabel.csv', index = False) # Storing to a data file

print("Correlations between the different variables:\n",file=sourceFile)
print(round(Kendall_corr,3).to_markdown(index = True, tablefmt="fancy_grid"),file=sourceFile)

## Step 2: Machine Learning setup

In [7]:
print("\n----------------------------------------------------------------------------------------------------------------------------\n",file=sourceFile)
print("Step 2 : Machine Learning setup\n".upper(),file=sourceFile)

# Defining the feature matrix and response vectors

X=data[['L1','L2','L3','R0','R1','R2','Gain','Phase']].values # Feature Matrix
y=data[['0_50','50_100','100_150','150_200','200_250','250_300','300_350','350_400','400_450','450_500']].values # Response vector

print(X.shape)
print(y.shape)

(551357, 8)
(551357, 10)


In [8]:
# Normalise the feature vector

X=preprocessing.StandardScaler().fit(X).transform(X)

In [9]:
# Train/test split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=3)
print('Size of training set Feature Matrix {}'.format(X_train.shape),file=sourceFile)
print('Size of testing set Feature Matrix {}'.format(X_test.shape),file=sourceFile)
print('Size of training set Feature Matrix {}'.format(X_train.shape))
print('Size of testing set Feature Matrix {}'.format(X_test.shape))

Size of training set Feature Matrix (441085, 8)
Size of testing set Feature Matrix (110272, 8)


## Step 3: K-nearest neighbours

In [10]:
print("\n----------------------------------------------------------------------------------------------------------------------------\n",file=sourceFile)
print("Step 3 : K-nearest neighbours\n".upper(),file=sourceFile)

## Building the K nearest neighbours (KNN) models

t = time.time()

Ks=50 # Maximum number of neighbours

parameters_Kneigh = [
    {
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': range(1,Ks+1),
    },
]

search_Kneigh = GridSearchCV(ClassifierChain(), parameters_Kneigh, scoring='accuracy', n_jobs=-1)
search_Kneigh.fit(X_train,y_train)

pred_Kneigh_test = search_Kneigh.predict(X_test)
mean_acc_test=metrics.accuracy_score(y_test,pred_Kneigh_test)

print("The highest testing accuracy is",round(100*mean_acc_test,3),"% obtained for",search_Kneigh.best_params_['classifier__n_neighbors'], "neighbour(s)",file=sourceFile)
print("\nElapsed time:",round((time.time() - t),2),"seconds",file=sourceFile)

print("The highest testing accuracy is",round(100*mean_acc_test,3),"% obtained for",search_Kneigh.best_params_['classifier__n_neighbors'], "neighbour(s)")
print("\nElapsed time:",round((time.time() - t),2),"seconds")

result_Kneigh=search_Kneigh.cv_results_['mean_test_score']
np.savetxt("Results_Kneigh_Multilabel.csv", result_Kneigh, delimiter=",") # Saving results to a separate file

The highest testing accuracy is 97.501 % obtained for 1 neighbour(s)

Elapsed time: 11162.94 seconds


## Step 4: Decision Trees

In [11]:
print("\n----------------------------------------------------------------------------------------------------------------------------\n",file=sourceFile)
print("Step 4 : Decision Trees\n".upper(),file=sourceFile)

## Building the Decision Trees models

t = time.time()

Ks=50 # Maximum number of layers

parameters_Tree = [
    {
        'classifier': [DecisionTreeClassifier(criterion="entropy")],
        'classifier__max_depth': range(1,Ks+1),
    },
]

search_Tree = GridSearchCV(ClassifierChain(), parameters_Tree, scoring='accuracy', n_jobs=-1)
search_Tree.fit(X_train,y_train)

pred_Tree_test = search_Tree.predict(X_test)
mean_acc_test=metrics.accuracy_score(y_test,pred_Tree_test)

print("The highest testing accuracy is",round(100*mean_acc_test,3),"% obtained for",search_Tree.best_params_['classifier__max_depth'], "layer(s)",file=sourceFile)
print("\nElapsed time:",round((time.time() - t),2),"seconds",file=sourceFile)

print("The highest testing accuracy is",round(100*mean_acc_test,3),"% obtained for",search_Tree.best_params_['classifier__max_depth'], "layer(s)")
print("\nElapsed time:",round((time.time() - t),2),"seconds")

result_Tree=search_Tree.cv_results_['mean_test_score']
np.savetxt("Results_Tree_Multilabel.csv", result_Tree, delimiter=",") # Saving results to a separate file

The highest testing accuracy is 97.628 % obtained for 37 layer(s)

Elapsed time: 849.19 seconds


## Step 5: Random forests

In [12]:
print("\n----------------------------------------------------------------------------------------------------------------------------\n",file=sourceFile)
print("Step 5 : Random forests\n".upper(),file=sourceFile)

## Building the Random forest models for various numbers of trees in the forest

t = time.time()

Ks=100 # Maximum number of trees in the forest

parameters_Forest = [
    {
        'classifier': [RandomForestClassifier(criterion='entropy')],
        'classifier__n_estimators': range(1,Ks+1),
    },
]

search_Forest = GridSearchCV(ClassifierChain(), parameters_Forest, scoring='accuracy', n_jobs=-1)
search_Forest.fit(X_train,y_train)

pred_Forest_test = search_Forest.predict(X_test)
mean_acc_test=metrics.accuracy_score(y_test,pred_Forest_test)

print("The highest testing accuracy is",round(100*mean_acc_test,3),"% obtained for",search_Forest.best_params_['classifier__n_estimators'], "tree(s)",file=sourceFile)
print("\nElapsed time:",round((time.time() - t),2),"seconds",file=sourceFile)

print("The highest testing accuracy is",round(100*mean_acc_test,3),"% obtained for",search_Forest.best_params_['classifier__n_estimators'], "tree(s)")
print("\nElapsed time:",round((time.time() - t),2),"seconds")

result_Forest=search_Forest.cv_results_['mean_test_score']
np.savetxt("Results_Forest_Multilabel.csv", result_Forest, delimiter=",") # Saving results to a separate file

/home/rgaudron/anaconda3/envs/JupyterLab_env/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


The highest testing accuracy is 98.352 % obtained for 99 tree(s)

Elapsed time: 23608.87 seconds


## Step 6: Multilayer Perceptrons

In [13]:
print("\n----------------------------------------------------------------------------------------------------------------------------\n",file=sourceFile)
print("Step 6 : Multi-layer Perceptrons\n".upper(),file=sourceFile)

## Building the Multi-layer Perceptrons models for various numbers of trees in the forest

t = time.time()

parameters_MLP = [
    {
        'classifier': [MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(100,100,100),
                                random_state=1, max_iter=10000, verbose=False, tol=1e-4, batch_size='auto')],
        'classifier__alpha': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3],  # Regularisation parameter
        'classifier__learning_rate_init': [0.0001, 0.0003, 0.001, 0.003, 0.006, 0.01], # Initial learning rate
    },
]

search_MLP = GridSearchCV(ClassifierChain(), parameters_MLP, scoring='accuracy', n_jobs=-1)
search_MLP.fit(X_train,y_train)

pred_MLP_test = search_MLP.predict(X_test)
mean_acc_test=metrics.accuracy_score(y_test,pred_MLP_test)

print("The highest testing accuracy is",round(100*mean_acc_test,3),"% obtained with alpha=", search_MLP.best_params_['classifier__alpha'], "and initial learning rate=", search_MLP.best_params_['classifier__learning_rate_init'],file=sourceFile)
print("\nElapsed time:",round((time.time() - t),2),"seconds",file=sourceFile)

print("The highest testing accuracy is",round(100*mean_acc_test,3),"% obtained with alpha=", search_MLP.best_params_['classifier__alpha'], "and initial learning rate=", search_MLP.best_params_['classifier__learning_rate_init'])
print("\nElapsed time:",round((time.time() - t),2),"seconds")

The highest testing accuracy is 94.205 % obtained with alpha= 0.001 and initial learning rate= 0.001

Elapsed time: 187386.37 seconds


## Step 7: Closing the file

In [14]:
sourceFile.close()